# ArXiv AI 논문 검색 시스템: RAG와 에이전틱 AI

이 튜토리얼에서는 다음을 배웁니다:
1. Qdrant 벡터 데이터베이스 설정
2. RAG (Retrieval-Augmented Generation) 구현
3. LangChain과 LangGraph를 사용한 멀티 에이전트 AI 시스템

## 1단계: 필요한 라이브러리 설치

필요한 패키지들을 설치합니다.

In [2]:
!pip install -q qdrant-client
!pip install -q transformers torch accelerate
!pip install -q sentence-transformers
!pip install -q pandas
!pip install -q langchain
!pip install -q langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.9/471.9 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.6 which is incompatible.


## 2단계: 라이브러리 임포트

프로젝트에 필요한 라이브러리들을 불러옵니다.

In [3]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List, Any, Optional
import operator
import warnings
warnings.filterwarnings('ignore')

## 3단계: 데이터 로드 및 탐색

ArXiv AI 논문 데이터셋을 로드하고 구조를 확인합니다.

In [4]:
df = pd.read_csv('arxiv_ai.csv')

print(f"전체 논문 개수: {len(df)}")
print(f"\n데이터셋 컬럼: {df.columns.tolist()}")
print(f"\n첫 번째 논문 예시:")
print(f"제목: {df.iloc[0]['title']}")
print(f"요약: {df.iloc[0]['summary'][:200]}...")

전체 논문 개수: 10000

데이터셋 컬럼: ['authors', 'categories', 'comment', 'doi', 'entry_id', 'journal_ref', 'pdf_url', 'primary_category', 'published', 'summary', 'title', 'updated']

첫 번째 논문 예시:
제목: Dynamic Backtracking
요약: Because of their occasional need to return to shallow points in a search
tree, existing backtracking methods can sometimes erase meaningful progress
toward solving a search problem. In this paper, we ...


## 4단계: 데이터 전처리

검색에 사용할 데이터를 정리합니다. 튜토리얼을 위해 500개의 논문만 사용합니다.

In [5]:
df = df[['title', 'summary', 'published', 'authors']].dropna()
df = df.sample(500, random_state=42).reset_index(drop=True)

df['published_year'] = pd.to_datetime(df['published']).dt.year
df['text'] = df['title'] + ' ' + df['summary']

print(f"전처리 후 논문 개수: {len(df)}")

전처리 후 논문 개수: 500


## 5단계: 임베딩 모델 로드

텍스트를 벡터로 변환하는 임베딩 모델을 로드합니다.

In [7]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

test_embedding = embedding_model.encode("test")
print(f"임베딩 차원: {len(test_embedding)}")

임베딩 차원: 384


## 6단계: Qdrant 벡터 데이터베이스 설정

Qdrant 클라이언트를 초기화하고 컬렉션을 생성합니다.

In [8]:
client = QdrantClient(":memory:")

collection_name = "arxiv_papers"

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

print(f"컬렉션 '{collection_name}' 생성 완료")

컬렉션 'arxiv_papers' 생성 완료


## 7단계: 논문 임베딩 및 벡터 DB에 저장

모든 논문을 벡터로 변환하여 Qdrant에 저장합니다.

In [9]:
points = []

for idx, row in df.iterrows():
    vector = embedding_model.encode(row['text']).tolist()

    point = PointStruct(
        id=idx,
        vector=vector,
        payload={
            "title": row['title'],
            "summary": row['summary'],
            "year": int(row['published_year']),
            "authors": row['authors']
        }
    )
    points.append(point)

    if (idx + 1) % 100 == 0:
        print(f"{idx + 1}개 논문 처리 완료")

client.upsert(collection_name=collection_name, points=points)
print(f"\n총 {len(points)}개 논문이 벡터 DB에 저장되었습니다.")

100개 논문 처리 완료
200개 논문 처리 완료
300개 논문 처리 완료
400개 논문 처리 완료
500개 논문 처리 완료

총 500개 논문이 벡터 DB에 저장되었습니다.


## 8단계: Granite 4.0 Micro 모델 로드

IBM의 Granite 4.0 Micro 언어 모델을 로드합니다.

In [10]:
model_path = "ibm-granite/granite-4.0-micro"

tokenizer = AutoTokenizer.from_pretrained(model_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
llm_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device if device == "cuda" else None
)
llm_model.eval()

print(f"모델이 {device}에 로드되었습니다.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

모델이 cuda에 로드되었습니다.


## 9단계: LLM 헬퍼 함수 생성

Granite 모델을 사용하여 텍스트를 생성하는 헬퍼 함수를 만듭니다.

In [11]:
def generate_response(prompt: str, max_tokens: int = 300) -> str:
    """Granite 모델을 사용하여 응답을 생성합니다."""
    chat = [{"role": "user", "content": prompt}]
    chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    input_tokens = tokenizer(chat_text, return_tensors="pt").to(device)

    output = llm_model.generate(**input_tokens, max_new_tokens=max_tokens)
    response = tokenizer.decode(output[0], skip_special_tokens=False)

    assistant_response = response.split("<|start_of_role|>assistant<|end_of_role|>")[-1]
    assistant_response = assistant_response.split("<|end_of_text|>")[0].strip()

    return assistant_response

print("LLM 헬퍼 함수 생성 완료")

LLM 헬퍼 함수 생성 완료


## 10단계: LangChain 도구 정의

에이전트가 사용할 도구들을 LangChain 형식으로 정의합니다.

In [12]:
@tool
def search_papers_tool(query: str, top_k: int = 3) -> str:
    """벡터 데이터베이스에서 쿼리와 관련된 논문을 검색합니다."""
    query_vector = embedding_model.encode(query).tolist()

    results = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        limit=top_k
    ).points

    output = f"검색 결과 ({len(results)}개 논문):\n\n"
    for i, result in enumerate(results, 1):
        output += f"[{i}] 제목: {result.payload['title']}\n"
        output += f"    연도: {result.payload['year']}\n"
        output += f"    요약: {result.payload['summary'][:200]}...\n"
        output += f"    유사도: {result.score:.3f}\n\n"

    return output

@tool
def summarize_papers_tool(topic: str) -> str:
    """특정 주제에 대한 논문들을 검색하고 심층 분석하여 요약합니다."""
    query_vector = embedding_model.encode(topic).tolist()

    results = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        limit=3
    ).points

    context = f"논문 분석 요청: {topic}\n\n"
    context += "검색된 논문들:\n"
    for i, result in enumerate(results, 1):
        context += f"\n[논문 {i}]\n"
        context += f"제목: {result.payload['title']}\n"
        context += f"출판연도: {result.payload['year']}\n"
        context += f"초록: {result.payload['summary'][:400]}\n"

    prompt = f"""{context}

위 논문들의 초록을 바탕으로 다음을 분석해주세요:

1. 핵심 연구 내용: 각 논문이 다루는 주요 기술과 방법론
2. 실용성 평가: 실제 프로젝트 적용 가능성
3. 추천 순위: 가장 유용한 논문부터 순서대로
4. 다음 단계: 이 논문들을 읽은 후 추가로 알아보면 좋을 주제

간결하고 실용적으로 답변해주세요."""

    summary = generate_response(prompt, max_tokens=400)

    return summary

tools = [search_papers_tool, summarize_papers_tool]
print("도구 정의 완료")

도구 정의 완료


## 11단계: RAG 파이프라인 구현

검색된 논문을 기반으로 답변을 생성하는 RAG 시스템을 구현합니다.

In [13]:
def rag_query(question, top_k=3):
    query_vector = embedding_model.encode(question).tolist()

    search_results = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        limit=top_k
    ).points

    context = "관련 논문들:\n\n"
    for i, result in enumerate(search_results, 1):
        context += f"[{i}] 제목: {result.payload['title']}\n"
        context += f"요약: {result.payload['summary'][:300]}...\n\n"

    prompt = f"{context}\n질문: {question}\n\n위 논문들을 바탕으로 간단히 답변해주세요."
    answer = generate_response(prompt)

    return answer, search_results

print("RAG 파이프라인 구현 완료")

RAG 파이프라인 구현 완료


## 12단계: LLM 단독 vs RAG 비교

먼저 RAG 없이 LLM만 사용했을 때와 RAG를 사용했을 때의 차이를 비교합니다.

In [14]:
question = "What are the latest techniques in reinforcement learning for robotics?"

print("=" * 70)
print("질문:", question)
print("=" * 70)

# 1. LLM만 사용 (RAG 없이)
print("\n[1] LLM 단독 답변 (검색 없이):")
print("-" * 70)
llm_only_answer = generate_response(question)
print(llm_only_answer)

# 2. RAG 사용 (검색 + LLM)
print("\n" + "=" * 70)
print("\n[2] RAG 답변 (논문 검색 + LLM):")
print("-" * 70)
rag_answer, papers = rag_query(question, top_k=3)

print("\n검색된 논문:")
for i, paper in enumerate(papers, 1):
    print(f"  [{i}] {paper.payload['title'][:80]}... ({paper.payload['year']})")

print("\nRAG 답변:")
print(rag_answer)

print("\n" + "=" * 70)
print("\n💡 차이점:")
print("- LLM 단독: 일반적인 지식만 사용 (환각 가능성 있음)")
print("- RAG: 실제 논문을 검색하여 근거 기반 답변 제공")
print("=" * 70)

질문: What are the latest techniques in reinforcement learning for robotics?

[1] LLM 단독 답변 (검색 없이):
----------------------------------------------------------------------
Reinforcement Learning (RL) is a rapidly evolving field, and there are several techniques that have been developed recently for robotics. Here are some of the latest:

1. Deep Reinforcement Learning (DRL): DRL combines reinforcement learning with deep learning, allowing robots to learn from large amounts of sensory data. This has been particularly useful in complex environments where traditional RL methods struggle.

2. Model-Based Reinforcement Learning: This approach involves building a model of the environment and using it to predict the outcomes of actions. This can lead to more efficient learning and better generalization to new situations.

3. Transfer Learning: This technique involves using knowledge gained from one task to improve learning in another related task. This can significantly reduce the amount of dat

## 14단계: 실전 프로젝트 질문으로 RAG 테스트

학생들이 실제 프로젝트에서 사용할 법한 구체적인 질문으로 테스트합니다.

In [15]:
question = "I want to build a recommender system for e-commerce. What papers should I read about collaborative filtering and deep learning approaches?"

answer, papers = rag_query(question, top_k=5)

print("프로젝트 질문:", question)
print("\n" + "="*50)
print("추천 논문:")
for i, paper in enumerate(papers, 1):
    print(f"\n[{i}] {paper.payload['title']}")
    print(f"    연도: {paper.payload['year']}")
    print(f"    유사도: {paper.score:.3f}")

print("\n" + "="*50)
print("프로젝트 가이드:")
print(answer)

프로젝트 질문: I want to build a recommender system for e-commerce. What papers should I read about collaborative filtering and deep learning approaches?

추천 논문:

[1] Triangle Graph Interest Network for Click-through Rate Prediction
    연도: 2022
    유사도: 0.355

[2] Uncertainty-aware Personal Assistant for Making Personalized Privacy Decisions
    연도: 2022
    유사도: 0.334

[3] A Recommender System based on Idiotypic Artificial Immune Networks
    연도: 2008
    유사도: 0.329

[4] Against Algorithmic Exploitation of Human Vulnerabilities
    연도: 2023
    유사도: 0.326

[5] Price Suggestion for Online Second-hand Items with Texts and Images
    연도: 2020
    유사도: 0.308

프로젝트 가이드:
E-commerce recommender systems can benefit from both collaborative filtering and deep learning approaches. Based on the provided papers, here are some key references you should consider:

1. Triangle Graph Interest Network for Click-through Rate Prediction: This paper focuses on click-through rate prediction using a Triangle Gra

## 14단계: LangGraph를 사용한 멀티 에이전트 상태 정의

에이전트의 상태를 정의합니다.

In [16]:
class AgentState(TypedDict):
    messages: Annotated[List, operator.add]
    query: str
    search_results: str
    summary: str
    next_step: str

print("에이전트 상태 정의 완료")

에이전트 상태 정의 완료


## 15단계: 에이전트 노드 함수들 구현

각 에이전트의 역할을 수행하는 노드 함수들을 구현합니다.

In [18]:
def search_agent(state: AgentState) -> AgentState:
    """논문 검색 에이전트"""
    query = state["query"]
    print(f"\n[검색 에이전트] '{query}' 검색 중...")

    result = search_papers_tool.invoke({"query": query, "top_k": 3})

    state["search_results"] = result
    state["next_step"] = "summarize"
    state["messages"].append(f"검색 완료: {query}")

    return state

def summarize_agent(state: AgentState) -> AgentState:
    """논문 요약 에이전트"""
    query = state["query"]
    print(f"\n[요약 에이전트] '{query}' 요약 중...")

    summary = summarize_papers_tool.invoke({"topic": query})

    state["summary"] = summary
    state["next_step"] = "end"
    state["messages"].append("요약 완료")

    return state

def router(state: AgentState) -> str:
    """다음 단계 결정"""
    if state["next_step"] == "summarize":
        return "summarize"
    return "end"

print("에이전트 노드 함수 구현 완료")

에이전트 노드 함수 구현 완료


## 16단계: LangGraph 워크플로우 구성

멀티 에이전트 워크플로우를 LangGraph로 구성합니다.

In [19]:
workflow = StateGraph(AgentState)

workflow.add_node("search", search_agent)
workflow.add_node("summarize", summarize_agent)

workflow.set_entry_point("search")

workflow.add_conditional_edges(
    "search",
    router,
    {
        "summarize": "summarize",
        "end": END
    }
)

workflow.add_edge("summarize", END)

app = workflow.compile()

print("멀티 에이전트 워크플로우 구성 완료")

멀티 에이전트 워크플로우 구성 완료


## 17단계: 멀티 에이전트 시스템 실행

구성한 멀티 에이전트 시스템을 실행합니다.

In [20]:
def run_multi_agent(query: str):
    initial_state = {
        "messages": [],
        "query": query,
        "search_results": "",
        "summary": "",
        "next_step": "search"
    }

    print("=" * 50)
    print(f"사용자 질문: {query}")
    print("=" * 50)

    result = app.invoke(initial_state)

    print("\n" + "=" * 50)
    print("검색 결과:")
    print(result["search_results"])

    print("=" * 50)
    print("최종 요약:")
    print(result["summary"])

    return result

result = run_multi_agent("reinforcement learning")

사용자 질문: reinforcement learning

[검색 에이전트] 'reinforcement learning' 검색 중...

[요약 에이전트] 'reinforcement learning' 요약 중...

검색 결과:
검색 결과 (3개 논문):

[1] 제목: Open Problem: Approximate Planning of POMDPs in the class of Memoryless Policies
    연도: 2016
    요약: Planning plays an important role in the broad class of decision theory.
Planning has drawn much attention in recent work in the robotics and sequential
decision making areas. Recently, Reinforcement L...
    유사도: 0.587

[2] 제목: Planning with RL and episodic-memory behavioral priors
    연도: 2022
    요약: The practical application of learning agents requires sample efficient and
interpretable algorithms. Learning from behavioral priors is a promising way to
bootstrap agents with a better-than-random ex...
    유사도: 0.581

[3] 제목: The Option-Critic Architecture
    연도: 2016
    요약: Temporal abstraction is key to scaling up learning and planning in
reinforcement learning. While planning with temporally extended actions is well
understood, crea

## 18단계: 실전 프로젝트 시나리오 테스트

다양한 프로젝트 상황에서 에이전트 시스템을 테스트합니다.

In [21]:
# 실전 프로젝트 질문들
test_queries = [
    "I need to build a chatbot with natural language understanding. What are the best papers on transformer-based dialogue systems?",
    "For my computer vision project on object detection in autonomous vehicles, which papers cover real-time detection methods?",
    "I'm working on a medical diagnosis system using neural networks. What research exists on interpretable AI for healthcare?"
]

for query in test_queries:
    print("\n\n" + "#" * 70)
    result = run_multi_agent(query)
    print("#" * 70)



######################################################################
사용자 질문: I need to build a chatbot with natural language understanding. What are the best papers on transformer-based dialogue systems?

[검색 에이전트] 'I need to build a chatbot with natural language understanding. What are the best papers on transformer-based dialogue systems?' 검색 중...

[요약 에이전트] 'I need to build a chatbot with natural language understanding. What are the best papers on transformer-based dialogue systems?' 요약 중...

검색 결과:
검색 결과 (3개 논문):

[1] 제목: Behaviour Trees for Creating Conversational Explanation Experiences
    연도: 2022
    요약: This paper presented an XAI system specification and an interactive dialogue
model to facilitate the creation of Explanation Experiences (EE). Such
specifications combine the knowledge of XAI, domain ...
    유사도: 0.529

[2] 제목: Decision-support for the Masses by Enabling Conversations with Open Data
    연도: 2018
    요약: Open data refers to data that is freely available for

## 결론

이 튜토리얼에서 배운 내용:
1. Qdrant를 사용한 벡터 데이터베이스 구축
2. 의미 기반 논문 검색 시스템 구현
3. RAG를 통한 검색 기반 답변 생성
4. LangChain 도구 정의 및 사용
5. LangGraph를 사용한 멀티 에이전트 워크플로우 구성
6. 검색 에이전트와 요약 에이전트의 협업

이제 여러분은 멀티 에이전트 기반 연구 논문 검색 시스템을 구축할 수 있습니다!